# Sonification, Chaos, and Fractals

The purpose of this notebook is twofold: to understand how [mapping](mapping.ipynb) can be applied to non-musical equations, and to demonstrate how *sonification* can reveal the behavior of non-musical functions.  The term sonification means to use an acoustic signal to perceive information inherent in a collection of data [Cramer]. Sonification is to the ear what graphic plotting is to the eye -- a way to understand relationships that would otherwise be difficult or impossible to perceive. Our ears are very keen instruments that can detect subtle, multidimensional changes in sound space. We are probably all familiar with listening to small fluxations in sound in order to detect the source of a mechanical problem. Sonification can yield information about regularity, overall shape, grouping, transitions, and deviations in data that present in an acoustic signal.

Running this notebook requires the musx package. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for directions on how to install musx in your environment.
<hr style="height:1px;color:gray">

Python imports:

In [ ]:
import sys
sys.path.append('/Users/taube/Software/musx')
from musx import Score, Note, Seq, MidiFile, setmidiplayer, version, playfile, rescale, \
    uniran, Choose, temper, scale, quantize, keynum
from musx.midi.gm import TaikoDrum, Celesta
print(f"musx version: {version}")

Generate midi files and automatically play them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal based midi player to use with musx.  If you dont have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /Users/taube/Music/SoundFonts/MuseScore_General.sf2")
print('OK!')

## Chaos

The term *deterministic chaos* refers to the behavior of systems that are not random but are nevertheless unpredictable in nature. The physical world is filled with complex dynamic processes such as turbulence and weather that are not random but whose behavior is nevertheless impossible to predict. There are several features that characterize these systems. First, chaotic systems are highly sensitive to initial conditions -- in one initial arrangement their elements produces periodicity; in another, closely proximate arrangement, complete unpredictability ensues. This is because chaotic system are nonlinear, which means that changes in one element do not produce a like, or equivalent change in others. Because of nonlinearity, cause and effect relationships are not proportional in a chaotic system, so that two functions for a single equation may evolve in a similar way but can then suddenly diverge into different paths:

<img src="support/butterfly.jpg">

Development of the logistic function for f0=.95 and f0=.949999999 (blue) for c=3.75 in both cases. This minimal variation of the initial value yields dramatically different sequences for n>30.

## The Logistic Map

The Logistic Map is a simple example of a discrete dynamical system that actually names a whole family of iterative functions described by the Logistic Equation:

<!-- fn + 1 = cfn(1 - fn); for 0.0 ≤ c ≤ 4.0 -->

<code>f<sub>n+1</sub> = cf<sub>n</sub>(1 - f<sub>n</sub>); for 0.0 ≤ c ≤ 4.0</code>

Each real value of c in the equation yields a different dynamic behavior and hence a different function.

As a model for the development of populations (and similar processes), the logistic map may be broken down into an activating and inhibiting term, related by a function R and with a weighting constant c applied to one of the terms:

`f=(c * activator) R inhibitor.`

In the case of the Logistic Map, R is multiplication:

`f=(c * activator) * inhibitor`

<img src="support/actinhib.jpg">

Activator term x (green) and inhibitor term 1-x (red) of the logistic map and their product (blue), which exhibits a global maximum of 0.25. To keep the final product c*x(1-x) stable between 0.0 and 1.0 it is required that 0≤c≤4.0.

### Bifurcation

The behavior of the logistic map does not depend on the value of f0 alone but also on the constant c. In fact, for numerous values of c, f0 doesn't matter much at all since the function will converge eventually. Most obviously, setting c=0 yields zero values for all values of fn for n>0. (Figure 4) Similar behavior results for all values of c<~3.0. At this point, the map converges to several attractors again, depending on the value of the initial f0. It converges first to 2, then to 4, 8, 16 ... attractors until it reaches chaotic behavior. (This repeated doubling within families of functions such as the logistic map has been first noted and extensively studied by the physicist Mitchell J. Feigenbaum.) Interestingly, the sequence of functions of the logistic map traverse several states of temporary tranquility on their way to sheer chaotic behavior . Thus, fn is actually a family of functions of the form y(s,c) = c * y * (1 - y), with seed values s = {0.0005, 0.001 ... 1.0}. Although the logistic map may give radically different numerical sequences for any two functions of the same family (functions with equal c but differing seed value s) the distribution pattern of a family after i iterations and the histogram of any of its functions up to the ith iteration approach the same attractor.

This animation shows a sequence of iterations from 0 through 560, using 1000 initial seed values. Note a) how each iteration adds a new sinusoidal branch to the attractor and b) how each new branch is limited by its neighboring branches. Note, too, how distinctly chaotic and non-chaotic areas of attraction coexist as the number of iterations increases. The repeated doubling of attractors within families of functions such as the logistic map was first noted and extensively studied by the physicist Mitchell J. Feigenbaum.

See this video:

In [ ]:
from IPython.display import Video
Video("support/bifurcation.mp4", embed=True)

### Mapping the Logistic Map

The logmap composer defined below maps values from the Logistic Map onto key numbers between *key1* and *key2*.  Initial conditions are represented by the parameter *c*, which may range
between 0 less than 4.0, and the initial value for *y*|:

In [ ]:
def logmap(score, c, y, num, rate, dur, key1, key2, amp, chan=0):
    for _ in range(num):
        k = rescale(y, 0, 1, key1, key2)
        m = Note(time=score.now, pitch=k, duration=dur, amplitude=amp, instrument=chan)
        score.add(m)
        y = (y * c * (1 - y))
        yield rate

print(f"logmap: {logmap}")

Listen to it:

In [ ]:
C, Y = 3.7, uniran() 
print(f"C: {C}, Y: {Y}")
score = Score(out=Seq())
score.compose( logmap(score, C, Y, 200, .125, .25, 60, 96, .6) )
file = MidiFile("sonification.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

Increase the chaos factor close to the maximum:

In [ ]:
C, Y = 3.99, uniran()
print(f"C: {C}, Y: {Y}")
score=Score(out=Seq())
score.compose(logmap(score, C, Y, 200, .125, .25, 60, 96, .6))
file = MidiFile("sonification.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

Listen to two voices diverging:

In [ ]:
C = 3.99
Y1 = .95
Y2 = .934567
print(f"C: {C}, Y1: {Y1}, Y2: {Y2}")

score = Score(out=Seq())
score.compose([logmap(score, C, Y1, 200, .125, .25, 60-24, 96-24, .6),
               logmap(score, C, Y2, 200, .125, .25, 60, 96, .6, chan=0)])
file = MidiFile("sonification.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

This example maps chaotic rhythms onto random choices from the midi drum map (channel 9):

In [ ]:
def groove(score, chaos, y, num, pulse):
    keynums = [between(35,82), between(35,82)]
    c = Choose(keynums)
    for _ in range(num):
        k = c.next()
        score.add(Note(time=score.now, pitch=k, duration=.01, amplitude=.75, instrument=9))
        yield pulse * y
        y = y * chaos * (1 - y)

C, Y = 3.99, uniran()
print("C:", C, ", Y:", Y)
score = Score(out=Seq())
score.compose(groove(score, C, Y, 50, .5))
file = MidiFile("sonification.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

Use a scale other than chromatic, for example three complete octaves of a Slendro scale (5-Tone Equal Temperment) starting on middle C:

In [ ]:
slendro = scale(60, 5 * 3 + 1, temper(2**(1/5)))
print(f"3-octave slendro scale:\n{slendro}")

In [ ]:
def logmapscale (score, y, c, num, rate, dur, scl, amp, chan):
    for _ in range(num):
        i = round(rescale(y, 0, 1, 0, len(scl)-1))
        k = scl[i]
        m = Note(time=score.now, duration=dur, pitch=k, amplitude=amp, instrument=chan)
        score.add(m)
        yield rate
        y = (y * c * (1 - y))

C, Y = 3.8, uniran()
print("C:", C, ", Y:", Y)

meta = MidiFile.metatrack(ins={0: TaikoDrum, 1: Celesta, 2: TaikoDrum, 3:Celesta}, 
                          microdivs=4)
score = Score(out=Seq())
score.compose(logmapscale(score, Y, C, 100, .125, .25, slendro, .6, 0))
file = MidiFile("sonification.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

This example links both rhythms and pitches together to form more distinctive sonic gestures. The rhythms are quantized to increments of .2:

In [ ]:
def logmap2d(q, chaos, y, num, pulse, key1, key2, dur):
    for _ in range(num):
        k = rescale(y, 0, 1, key1, key2)
        d = quantize(rescale(y, 0, 1, 0, pulse), .2)
        score.add(Note(time=score.now, pitch=k, duration=dur, amplitude=.5))
        yield d
        y = y * chaos * (1 - y)
        
C, Y = 3.99, uniran()
print("C:", C, ", Y:", Y)

meta = MidiFile.metatrack(ins={0: TaikoDrum, 1: TaikoDrum, 2: Celesta, 3: Celesta}, 
                          microdivs=4)
score = Score(out=Seq())
score.compose(logmap2d(score, C, Y, 200, .25, 60, 96, 1))
file = MidiFile("sonification.mid", [meta, score.out]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)


### Henon Map

The formula for calculating values in the Henon Map is:

`x[n+1] = 1 + a*x[n]^2 +  b*y[n]`

`y[n+1] = x[n]`


<img src="support/henon.jpg"/>

We can define a function that does this for us, it takes an old x y "point" (a list of 2 numbers) and returns a  new point representing the new x and y values


In [ ]:
def henon(point, alpha, beta):
    oldx = point[0]
    oldy = point[1]
    newx = 1 + (alpha * oldx * oldx) + (beta * oldy)
    newy = oldx
    return [newx, newy]

print(f"henon: {henon}")

Some choices for alpha and beta:

| a        | b        | x        | y       |
|----------|----------|----------|---------|
| -1.56693 | 0.003937 | 0.189406 | 0.18940 |
| 0.42     | -0.999   |          |         |
| 0.22     | -0.999   |          |         |
| 0.22     | -1.0     |          |         |
| -0.989   | 0.51     |          |         |
| -1.191   | 0.31     |          |         |
| -1.595   | 0.21     |          |         |
| -4       | 0.3      |          |         |

<!--
| a        	| b        	| x        	| y       	|
| ---------	| ---------	| ---------	| --------	|
| -1.56693 	| 0.003937 	| 0.189406 	| 0.18940 	|
| 0.42     	| -0.999   	|          	|         	|
| 0.22     	| -0.999   	|          	|         	|
| 0.22     	| -1.0     	|          	|         	|
| -0.989   	| 0.51     	|          	|         	|
| -1.191   	| 0.31     	|          	|         	|
| -1.595   	| 0.21     	|          	|         	|
| -1.4     	| 0.3      	|          	|         	|
-->

In [ ]:
def playhemap(score, num, rhy, dur, lok, hik, a, b, x, y, amp):
    point = [x,y]
    for _ in range(num):
        k = rescale( point[0], -1, 1, lok, hik)
        m = Note(time=score.now, duration=dur, amplitude=amp, pitch=k)
        score.add(m)
        yield rhy
        point = henon(point, a, b)

print(f'playhemap: {playhemap}')

In [ ]:
score = Score(out=Seq())
score.compose( playhemap(score, 100, .125, .25, 30, 100, 0.42, -.999, 0.0, 0.0, .7) )
file = MidiFile("sonification.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

playhemap2 is similar to playhemap except it plays notes that lie within the
low/high keynum range and pauses (rests) for note outside the range. This give the algorithm some rhythmic interest.

In [ ]:
def playhemap2(score, num, rhy, dur, lok, hik, a, b, x, y, amp):
    poi = [x,y]
    for _ in range(num):
        k = rescale( poi[0], -1, 1, 0, 127)
        if k >= lok and k <= hik:
            m = Note(time=score.now, duration=dur, amplitude=amp, pitch=k)
            score.add(m)
        yield rhy
        poi = henon(poi,a,b)

print(f'playhemap2: {playhemap2}')

In [ ]:
score = Score(out=Seq())
A, B = -1.56693, 0.003937
score.compose( playhemap2(score, 200, .1, .21, 30, 100, A, B, X, Y, .7) )
file = MidiFile("sonification.mid", score.out).write()
print(f"A: {A}, B: {B}, X: {X}, Y: {Y}")
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

## Fractals

### Sierpinksi's Triangle

Sierpinksi's Triangle is a famous two-dimensional fractal. The rules to create Sierpinski's triangle are simple:

* Start with a triangle.
* Draw a smaller triangle in the middle of the larger triangle with the points of the smaller triangle on the midpoints of the larger triangle's sides. This partitions the large triangle into 4 smaller triangles: a middle triangle and the triangles at each of the corners of the larger triangle.
* Leave the middle triangle empty.
* Visit each of the corner triangles in the larger triangle and repeat the rules starting with step 1 but now using each of these smaller triangles as the starting triangle.

Repeat for as many levels as you want: the result is Sierpinski's Triangle.

<img src="support/sierp.png"/>

Here is a musical “homage” to the Sierpinski triangle using a recursive process to generate a self-similar melody based on a set set of 3 tones representing the "sides" of the triangle:

<img src="support/map-7.png"/>

In [ ]:
def sierpinski(score, tone, shape, trans, levels, dur, amp):
    num = len(shape)
    for i in shape:
        k = tone + i
        # play current tone in melody
        m = Note(time=score.now, duration=dur, pitch=min(k,127),
                 amplitude=amp, instrument=0)
        score.add(m)
        if (levels > 1):
            # sprout melody on tone at next level
            score.compose(sierpinski(score, (k + trans), shape,
                            trans, levels - 1, dur / num,  amp))
        yield dur

print(f"sierpinski: {sierpinski}")

Note: Specify levels and melody length with care! The number of events sierpinski generates is exponentially related to the length of the melody and the number of levels. For example the first compose() generates 120 events, the second 726 and the third 2728!

In [ ]:
score = Score(Seq())

score.compose(sierpinski(score, keynum('a0'), [0, 11, 3], 12, 4, 3, .5))
#score.compose(sierpinski(score, keynum('a0'), [0, 7, 5], 8, 5, 7, .5))
#score.compose(sierpinski(score, keynum('a0'), [0, -1, 2, 13], 12, 5, 24, .5))

file = MidiFile("sonification.mid", score.out).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)

## Just for fun

* Start with a positive integer.
* If it is odd, multiply it by 3 and add 1.
* If it is even, divide it by 2.
* Apply the same two rules to the new value and continue.

No matter what number you start with, the system seems to eventually settle down to an infinite loop 4, 2, 1... BUT no one has been able to prove this conjecture is true.

Define a function or generator `three_x_plus_1(num, stop=None)` that produces a sequence of 3x+1 numbers until it reaches the 4 2 1 cycle, or optionally terminates after `stop` number of repetitions.   Given this stream of numbers, map the values to musical material according to rules you derive or by adopting some of the mapping and transformation techniques outlined in this lesson and the previous notebooks.

For more information on 3x+1 see the video: [The Simplest Math Problem No One Can Solve](https://www.youtube.com/watch?v=094y1Z2wpJg)
